In [ ]:
from openai import OpenAI

client = OpenAI(api_key="<api key>")

In [26]:
file = client.files.create(
    file=open('./json/taoyuan_airport_datasets.jsonl', "rb"),
    purpose='fine-tune'
)
print(file)

FileObject(id='file-wz6ezJ2uhnCSoV9LoHApQve6', bytes=39480, created_at=1709265638, filename='taoyuan_airport_datasets.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [27]:
fine_tuning_response = client.fine_tuning.jobs.create(
    training_file=file.id,
    model="gpt-3.5-turbo-0125"
)
print(fine_tuning_response)

FineTuningJob(id='ftjob-NgRByricOvNRkuRYdzqA78Fo', created_at=1709265681, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Hvc5Ktuhn0H5KtvCPdT24IAS', result_files=[], status='validating_files', trained_tokens=None, training_file='file-wz6ezJ2uhnCSoV9LoHApQve6', validation_file=None, user_provided_suffix=None)


In [39]:
fine_tuning_job_list = client.fine_tuning.jobs.list()
print(fine_tuning_job_list)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-NgRByricOvNRkuRYdzqA78Fo', created_at=1709265681, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:firebird-company::8xpDPTdV', finished_at=1709268914, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Hvc5Ktuhn0H5KtvCPdT24IAS', result_files=['file-L1hUvUQpFIHujyUByytYsatG'], status='succeeded', trained_tokens=50697, training_file='file-wz6ezJ2uhnCSoV9LoHApQve6', validation_file=None, user_provided_suffix=None)], object='list', has_more=False)


In [29]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": "你是你是一個機場的智能客服。"},
        {"role": "user", "content": "請問轉機時間較久，那裡可以休息?"}
    ],
    temperature=0
)

print(completion.choices[0].message.content)

您好！在機場內通常會有休息區、休息室或休息椅供旅客使用。您可以前往機場的休息區或休息室，或者在候機廳內找尋舒適的座位休息。另外，有些機場也設有休息室或酒店供旅客短暫休息。您可以向機場工作人員詢問有關休息設施的資訊。祝您旅途愉快！


In [31]:
completion = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:firebird-company::8xpDPTdV",
    messages=[
        {"role": "system", "content": "你是你是一個機場的智能客服。"},
        {"role": "user", "content": "請問轉機時間較久，那裡可以休息?"}
    ],
    temperature=0
)

print(completion.choices[0].message.content)

轉機旅客可休息區域：1. 第一航廈管制區內2樓抵達層轉機旅客休息區。2. 第二航廈轉機桃園市旅館公會協會3. 第二航廈管制區內3樓出發層轉機旅客休息區。3. 第一、第二航廈管制區內4樓出發層環亞付費貴賓室。4. 第一航廈管制區內4樓出發層轉機旅客免費休息區。5. 第二航廈管制區內4樓出發層轉機旅客休息區。


In [33]:
client.fine_tuning.jobs.list()

openai.pagination.SyncCursorPage[FineTuningJob]

# Fine-tuning data format validation
- https://cookbook.openai.com/examples/chat_finetuning_data_prep

In [17]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [18]:
data_path = './json/taoyuan_airport_datasets.jsonl'

# Load the dataset
with open(data_path, 'r') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 81
First example:
{'role': 'system', 'content': '你是一個機場的智能客服。'}
{'role': 'user', 'content': '請問動物申請轉機需要哪些文件？'}
{'role': 'assistant', 'content': '1. 攜帶活體動物出境需符合入境國相關檢疫規定，請先洽出境國檢疫單位確認完成相關檢疫申請程序。2. 有攜帶活體動物轉機需求時，亦需經本國檢疫署進行轉機檢查，建議可透過航空公司協助提出申請。'}


In [19]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [20]:
encoding = tiktoken.get_encoding("cl100k_base")
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [21]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 83, 411
mean / median: 212.30864197530863, 213.0
p5 / p95: 99.0, 403.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 26, 340
mean / median: 153.0, 154.0
p5 / p95: 26.0, 340.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [22]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~17197 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~51591 tokens
